In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
import glob
import os
import pickle
import json

import cv2
import numpy as np

In [3]:
from skimage.feature import hog

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score, make_scorer

In [4]:
DATA_DIR = os.path.join('../data')
VEHICLES = os.path.join(DATA_DIR, 'vehicles')
NON_VEHICLES = os.path.join(DATA_DIR, 'non-vehicles')

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV

In [6]:
class HogFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, color_scheme, orientation, pixels_per_cell, cells_per_block):
        self.color_scheme = color_scheme
        self.orientation = orientation
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
    
    def _hog_feature(self, image):
        return hog(image, orientations=self.orientation, 
                   pixels_per_cell=(self.pixels_per_cell, self.pixels_per_cell), 
                   cells_per_block=(self.cells_per_block, self.cells_per_block), 
                   feature_vector=True)
    
    def fit(self, X, y, **fit_params):
        return self
    
    def transform(self, X):
        X_new = []
        for x in X:
            img = cv2.cvtColor(x, self.color_scheme).astype(np.float32)/255
            # HOG Features
            rhf = self._hog_feature(img[:, :, 0])
            ghf = self._hog_feature(img[:, :, 1])
            bhf = self._hog_feature(img[:, :, 2])
            
            # Histogram Features
            channel1_hist = np.histogram(img[:,:,0], bins=32)
            channel2_hist = np.histogram(img[:,:,1], bins=32)
            channel3_hist = np.histogram(img[:,:,2], bins=32)
            # Concatenate the histograms into a single feature vector
            hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
            
            X_new.append(np.hstack((rhf, ghf, bhf, hist_features)))
        return np.array(X_new)
    
    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X, y).transform(X)

In [7]:
def read(data_dir):
    for img_path in glob.iglob(os.path.join(data_dir, '*', '*.png')):
        yield cv2.imread(img_path)

In [8]:
vehicles = [feature for feature in read(VEHICLES)]

In [9]:
non_vehicles = [feature for feature in read(NON_VEHICLES)]

In [10]:
data = np.vstack((vehicles, non_vehicles))
labels = np.hstack((np.ones(len(vehicles), np.int32), np.zeros(len(non_vehicles), np.int32)))

In [11]:
len(labels) == len(data)

True

In [12]:
data.shape

(17760, 64, 64, 3)

In [13]:
labels.shape

(17760,)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [15]:
x_train.shape

(14208, 64, 64, 3)

In [16]:
clf = svm.LinearSVC()
scaler = StandardScaler()
extractor = HogFeatureExtractor(cv2.COLOR_BGR2YCrCb, 9, 8, 2)
pipeline = Pipeline([('hog', extractor), ('scaler', scaler), ('svc', clf)])
# pipeline.fit(x_train, y_train)

In [17]:
color_schemes = [cv2.COLOR_BGR2RGB, cv2.COLOR_BGR2HLS, cv2.COLOR_BGR2YCrCb]
orientations = [9, 11, 13]
ppc = [8, 16]
cpb = [2, 3]
param_grid = dict(hog__color_scheme=color_schemes,
                  hog__orientation=orientations, 
                  hog__pixels_per_cell=ppc, 
                  hog__cells_per_block=cpb)

In [18]:
grid = GridSearchCV(pipeline, param_grid=param_grid, 
                    scoring=make_scorer(f1_score))

In [19]:
grid.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('hog', HogFeatureExtractor(cells_per_block=2, color_scheme=36, orientation=9,
          pixels_per_cell=8)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'hog__color_scheme': [4, 52, 36], 'hog__orientation': [9, 11, 13], 'hog__pixels_per_cell': [8, 16], 'hog__cells_per_block': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(f1_score), verbose=0)

In [20]:
with open('grid_cv_hist.p', 'wb') as _file:
    pickle.dump(grid, _file)

In [21]:
grid.best_params_

{'hog__cells_per_block': 2,
 'hog__color_scheme': 36,
 'hog__orientation': 13,
 'hog__pixels_per_cell': 8}

In [22]:
grid.best_score_

0.99198950435701028

In [23]:
import pandas as pd

In [24]:
results = pd.DataFrame(grid.cv_results_)

In [25]:
results

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_hog__cells_per_block,param_hog__color_scheme,param_hog__orientation,param_hog__pixels_per_cell,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,66.335829,23.505637,0.976151,1.000000,2,4,9,8,"{'hog__color_scheme': 4, 'hog__orientation': 9...",26,0.975631,1.000000,0.976058,1.000000,0.976764,1.000000,3.618995,1.038284,0.000467,0.000000
1,32.127077,15.349342,0.969840,0.999540,2,4,9,16,"{'hog__color_scheme': 4, 'hog__orientation': 9...",33,0.968314,0.999788,0.972010,0.999576,0.969195,0.999258,0.262920,0.856850,0.001576,0.000218
2,75.356264,26.813124,0.974732,1.000000,2,4,11,8,"{'hog__color_scheme': 4, 'hog__orientation': 1...",29,0.972074,1.000000,0.974675,1.000000,0.977447,1.000000,3.692100,0.323568,0.002194,0.000000
3,33.865262,16.088247,0.972904,0.999717,2,4,11,16,"{'hog__color_scheme': 4, 'hog__orientation': 1...",32,0.971792,0.999682,0.973203,0.999470,0.973718,1.000000,1.075577,1.441640,0.000814,0.000218
4,81.255457,26.261958,0.976717,1.000000,2,4,13,8,"{'hog__color_scheme': 4, 'hog__orientation': 1...",25,0.977181,1.000000,0.975891,1.000000,0.977080,1.000000,4.981679,1.616481,0.000586,0.000000
5,36.641641,15.542252,0.974696,0.999965,2,4,13,16,"{'hog__color_scheme': 4, 'hog__orientation': 1...",31,0.973891,1.000000,0.976024,1.000000,0.974174,0.999894,1.489835,0.958747,0.000946,0.000050
6,60.745514,26.099978,0.988252,1.000000,2,52,9,8,"{'hog__color_scheme': 52, 'hog__orientation': ...",13,0.985993,1.000000,0.988530,1.000000,0.990234,1.000000,5.912787,1.722663,0.001742,0.000000
7,35.604699,15.698892,0.985893,1.000000,2,52,9,16,"{'hog__color_scheme': 52, 'hog__orientation': ...",21,0.984954,1.000000,0.986475,1.000000,0.986249,1.000000,2.350016,0.884452,0.000670,0.000000
8,63.713528,23.782884,0.988455,1.000000,2,52,11,8,"{'hog__color_scheme': 52, 'hog__orientation': ...",12,0.987896,1.000000,0.987875,1.000000,0.989594,1.000000,2.268968,1.674660,0.000806,0.000000
9,30.375950,14.152921,0.987495,1.000000,2,52,11,16,"{'hog__color_scheme': 52, 'hog__orientation': ...",16,0.986644,1.000000,0.988151,1.000000,0.987691,1.000000,0.216579,0.114652,0.000630,0.000000


In [39]:
acc = grid.best_estimator_.score(x_test, y_test)

In [40]:
acc

0.99380630630630629

In [41]:
pred = grid.best_estimator_.predict(x_test)

In [42]:
cm = confusion_matrix(y_test, pred)

In [43]:
cm

array([[1821,    8],
       [  14, 1709]])

In [31]:
with open('classifier_hist.p', 'wb') as _file:
    pickle.dump(grid.best_estimator_, _file)

In [32]:
grid.best_estimator_

Pipeline(memory=None,
     steps=[('hog', HogFeatureExtractor(cells_per_block=2, color_scheme=36, orientation=13,
          pixels_per_cell=8)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [46]:
def hog_feature(image):
    return hog(image, orientations=13, 
               pixels_per_cell=(8, 8), 
               cells_per_block=(2, 2), 
               feature_vector=True)
    
def feature_extractor(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb).astype(np.float32)/255
    # HOG Features
    rhf = hog_feature(img[:, :, 0])
    ghf = hog_feature(img[:, :, 1])
    bhf = hog_feature(img[:, :, 2])
    return np.hstack((rhf, ghf, bhf))

def feature_extractor_w_hist(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb).astype(np.float32)/255
    # HOG Features
    rhf = hog_feature(img[:, :, 0])
    ghf = hog_feature(img[:, :, 1])
    bhf = hog_feature(img[:, :, 2])
    # Histogram Features
    channel1_hist = np.histogram(img[:,:,0], bins=32)
    channel2_hist = np.histogram(img[:,:,1], bins=32)
    channel3_hist = np.histogram(img[:,:,2], bins=32)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    return np.hstack((rhf, ghf, bhf, hist_features))
            

In [47]:
clf = svm.LinearSVC()
scaler = StandardScaler()
pipeline = Pipeline([('scaler', scaler), ('svc', clf)])

In [48]:
x_feat1 = [feature_extractor(x) for x in x_train]
x_feat2 = [feature_extractor_w_hist(x) for x in x_train]

In [49]:
pipeline.fit(x_feat1, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [50]:
x_test_feat1 = [feature_extractor(x) for x in x_test]
x_test_feat2 = [feature_extractor_w_hist(x) for x in x_test]

In [51]:
pred = pipeline.predict(x_test_feat1)

In [52]:
cm = confusion_matrix(pred, y_test)
cm

array([[1816,   27],
       [  13, 1696]])

In [53]:
pipeline.fit(x_feat2, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [54]:
pred = pipeline.predict(x_test_feat2)

In [55]:
cm = confusion_matrix(pred, y_test)
cm

array([[1821,   14],
       [   8, 1709]])

In [56]:
with open('classifier.p', 'wb') as _file:
    pickle.dump(pipeline, _file)